<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/twelvelabs-io/twelvelabs-developer-experience/blob/main/quickstarts/1.0.0-beta/TwelveLabs_Quickstart_Search.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in  Colab</a>
  </td>
</table>

# Search

This guide shows how to utilize the TwelveLabs Python SDK for searching within your video content.

## Key concepts

- **Index**: A container that organizes your video content
- **Asset**: Your uploaded file
- **Indexed asset**: A video that has been indexed and is ready for downstream tasks

## How it works

To search your videos, you must first upload and index them. The platform indexes videos asynchronously. After indexing completes, you can search your videos. Search results show video segments that match your search terms.

The upload method in this guide allows for files up to 4 GB when using publicly accessible URLs and 200 MB for local files. For details about the available upload methods and the corresponding limits, see the [Upload methods](https://docs.twelvelabs.io/docs/concepts/upload-methods) page.


**Types of search queries**

The platform supports three types of search queries:
- **Text queries**: Search using natural language descriptions of visual elements, actions, sounds, or spoken words
- **Image queries**: Search using images to find visually similar content in your videos
- **Composed queries**: Combine text descriptions with images for more precise results (Marengo 3.0 only)

For guidance on choosing the correct query type, see the [Search with text, image, and composed queries](https://docs.twelvelabs.io/docs/guides/search/search-with-text-and-image-queries) page.

**Search scope**

You can search within a single index per request. You cannot search at the video level or across multiple indexes simultaneously.

**Customize your search**

You can customize your search in the following ways:
- Specify which modalities to use: visual, audio, or transcription (spoken words)
- Choose how to combine modalities: use the or or and operators
- For searches within spoken words, select the match type: lexical, semantic, or both

# Prerequisites

Before you begin, ensure the following prerequisites are met:

- [Sign up](https://playground.twelvelabs.io/) for a free account and obtain your API key from the [API Key](https://playground.twelvelabs.io/dashboard/api-key) page. No credit card is required to use the Free plan. This plan allows you to index up to 600 minutes of videos, which is sufficient for a small project.
- Your video files must meet the [format requirements](https://docs.twelvelabs.io/docs/concepts/models/marengo#input-requirements).



# Procedure

## Install the TwelveLabs Python SDK

In [ ]:
%pip install twelvelabs

## Import the required packages

In [ ]:
import time
from twelvelabs import TwelveLabs

## Configure your API key


In [ ]:
# For Google Colab, store your API key as a Secret named `TL_API_KEY`. If you don't know how to create a Colab Secret, see https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75.

from google.colab import userdata
TL_API_KEY = userdata.get("TL_API_KEY")

# For other Python environments, you can use environment variables
# TL_API_KEY = os.environ.get('TL_API_KEY')

## Create an index

Indexes store and organize your video data, allowing you to group related videos. This guide shows how to create one, but you can also use an existing index. See the [Indexes](https://docs.twelvelabs.io/docs/concepts/indexes) page for more details on creating an index and specifying the model configuration.

In [ ]:
client = TwelveLabs(api_key=TL_API_KEY)

index = client.indexes.create(
    index_name="<YOUR_INDEX_NAME>",
    models=[{"model_name": "marengo3.0", "model_options": ["visual", "audio"]}]
)
if not index.id:
    raise RuntimeError("Failed to create an index.")
print(f"Created index: id={index.id}")

## Upload a video

In [ ]:
asset = client.assets.create(
    method="url",
    url="<YOUR_VIDEO_URL>" # Example: https://github.com/twelvelabs-io/twelvelabs-developer-experience/raw/refs/heads/main/quickstarts/steve_jobs_introduces_iphone_in_2007.mp4
    # Or use method="direct" and file=open("<PATH_TO_VIDEO_FILE>", "rb") to upload a file from the local file system
)
print(f"Created asset: id={asset.id}")

## Index your video

Index your video by adding the asset created in the previous step to an index. This operation is asynchronous.

In [ ]:
indexed_asset = client.indexes.indexed_assets.create(
    index_id=index.id,
    asset_id=asset.id,
    # enable_video_stream=True
)
print(f"Created indexed asset: id={indexed_asset.id}")

## Monitor the indexing process

The platform requires some time to index videos. Check the status of the indexing process until it’s completed.

In [ ]:
print("Waiting for indexing to complete.")
while True:
    indexed_asset = client.indexes.indexed_assets.retrieve(
        index_id=index.id,
        indexed_asset_id=indexed_asset.id
    )
    print(f"  Status={indexed_asset.status}")
    if indexed_asset.status == "ready":
        print("Indexing complete!")
        break
    elif indexed_asset.status == "failed":
        raise RuntimeError("Indexing failed")
    time.sleep(5)

## Perform searches

Perform a search within your index using a text or image query or a combination of both.

**Using text queries**:

In [ ]:
search_results = client.search.query(
    index_id=index.id,
    query_text="<YOUR_QUERY>", # Example: "Steve Jobs"
    search_options=["visual", "audio"]
    # operator="or" # Optional: Use "and" to find segments matching all modalities
    # transcription_options=["lexical", "semantic"]  # Optional: Control transcription matching (Marengo 3.0 only, requires "transcription" in search_options)
)

print("\nSearch results:")
print("Each result shows a video clip that matches your query:\n")
for i, clip in enumerate(search_results):
    print(f"Result {i + 1}:")
    print(f"  Video ID: {clip.video_id}")  # Unique identifier of the video
    print(f"  Rank: {clip.rank}")  # Relevance ranking (1 = most relevant)
    print(f"  Time: {clip.start}s - {clip.end}s")  # When this moment occurs in the video
    print()

**Using image queries**

In [ ]:
search_results = client.search.query(
    index_id=index.id,
    search_options=["visual"],
    query_media_type="image",
    query_media_url="<YOUR_IMAGE_URL>"
    # Or for a local file: query_media_file =open("<PATH_TO_IMAGE_FILE>", "rb")
)

**Using composed text and image queries**

In [ ]:
search_results = client.search.query(
    index_id=index.id,
    search_options=["visual"],
    query_text="<YOUR_QUERY>",
    query_media_type="image",
    query_media_url="<YOUR_IMAGE_URL>"
    # Or for a local file: query_media_file =open("<PATH_TO_IMAGE_FILE>", "rb")
)

# Next steps

For a comprehensive guide, see the [Search](https://docs.twelvelabs.io/docs/guides/search) page.